<a href="https://colab.research.google.com/github/Vincent-ondeng/Final_year_research_project/blob/main/Notebook/web_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium
!pip install geopandas
!pip install pandas
!pip install numpy

In [ ]:
#Import Libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip


In [ ]:
# Read the GeoJSON file using Geopandas
geojson = gpd.read_file("/content/wards.geojson")
geojson=geojson[['wardcode','geometry']] #only select 'wardcode' (Ward_Code) and 'geometry' columns
# Read the Facilities GeoJSON file using GeoPandas
geojson_f = gpd.read_file("/content/facilities.geojson")
geojson_f=geojson_f[['FID_', 'geometry']] #only select the FID_ and the geometry column

In [ ]:
#Read the covid-19 data using pandas
malaria_df=pd.read_csv("/content/wards.csv")
facilities_df=pd.read_csv("/content/facilities.csv")

In [ ]:
malaria_df.info()

In [ ]:
facilities_df.info()

In [ ]:
facilities_df.head()

In [ ]:
geojson.head()

In [ ]:
geojson_f.head()

In [ ]:
#Join the geojson file with malaria_df
df_final = geojson.merge(malaria_df, left_on="wardcode", right_on="wardcode", how="outer")

In [ ]:
#Join the geojson_f with the facilities_df dataframe
df_final2 = geojson_f.merge(facilities_df, left_on="FID_", right_on="FID", how="outer")

In [ ]:
df_final.head()

In [ ]:
df_final2.head()

In [ ]:
# Create a map instance
m = folium.Map(location=[-0.15386345206107715, 34.86414568574279],
               zoom_start=10,
               control_scale=True,
               prefer_canvas=True)
m

In [ ]:
folium.Choropleth(
    geo_data=geojson,
    name="Malaria Distribution by ward",
    data=malaria_df,
    columns=['wardcode', 'Total_co'],
    key_on='feature.properties.wardcode',
    nan_fill_color="White", #Use white color if there is no data available for the county
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Confirmed Malaria  Cases in Kisumu",
    line_color='black',
    highlight=True,
).add_to(m)
m

In [ ]:
#Add Customized Tooltips to the map
folium.features.GeoJson(
                    data=df_final,
                    name='Total Confirmed cases of Malaria in Kisumu',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['wardcode',
                                'ward',
                                'Confirme',
                                'Confirm1',
                                'Malaria_',
                                'Total_co'
                               ],
                        aliases=["WardCode:",
                                 "Ward Name:",
                                 "Confirmed Cases in Children under agve 5:",
                                 "Confirmed Cases in persons greater than age 5:",
                                 "Confirmed Cases in expectant women:",
                                 "Total Confirmed Malaria Cases:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(m)   

m

In [ ]:
# Adding the tile control layer for the different tiles in Kisumu County
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.TileLayer('Stamen terrain').add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
m.save('index.html')